<a href="https://colab.research.google.com/github/ebamberg/research-projects-ml/blob/main/examples/visualization/embedding_clusters_with_umap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install umap-learn plotly pandas numpy sentence-transformers hdbscan --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.7 MB/s eta 0:00:00


In [4]:
!curl -L -o ./news-headlines-2024.zip https://www.kaggle.com/api/v1/datasets/download/dylanjcastillo/news-headlines-2024

!unzip ./news-headlines-2024.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  225k  100  225k    0     0   478k      0 --:--:-- --:--:-- --:--:--  478k
Archive:  ./news-headlines-2024.zip
  inflating: news_data_dedup.csv     


In [6]:
import pandas as pd

df = pd.read_csv("./news_data_dedup.csv")
docs = [
    f"{title}\n{description}"
    for title, description in zip(df.title, df.description)
]

texts=[
    "King Charles sits on the throws",
    "The cat chasing the bird",
    "Queen Elizabeth is death.",
    "The bird sits on a tree",
]

In [9]:
from google.colab import userdata
from sentence_transformers import SentenceTransformer
from umap import UMAP
import pandas as pd
import numpy as np
import plotly.express as px
import hdbscan

model_id = "sentence-transformers/all-MiniLM-L6-v2"
hf_token = userdata.get('HF_TOKEN')

model = SentenceTransformer(model_id)

embeddings = model.encode(docs)

embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

array([[ 0.01299997, -0.02001937,  0.03787312, ..., -0.0176036 ,
         0.08934783, -0.06768813],
       [-0.00416241,  0.03095506, -0.04282113, ..., -0.02408291,
        -0.04449849, -0.03713955],
       [-0.02172824, -0.00271914,  0.10941687, ...,  0.00217858,
         0.01097496, -0.09007304],
       ...,
       [ 0.08573326,  0.04818096,  0.08196032, ...,  0.04652384,
        -0.06448124,  0.02147194],
       [ 0.08200873, -0.00633857,  0.06603416, ...,  0.00668818,
        -0.03189664, -0.0004388 ],
       [ 0.06723553,  0.08093952, -0.01455511, ..., -0.01280489,
        -0.00361777,  0.06653812]], dtype=float32)

In [10]:
hdb = hdbscan.HDBSCAN(min_samples=3, min_cluster_size=3).fit(embeddings)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [11]:
umap = UMAP(n_components=2, random_state=42, n_neighbors=80, min_dist=0.1)

df_umap = (
    pd.DataFrame(umap.fit_transform(np.array(embeddings)), columns=['x', 'y'])
    .assign(cluster=lambda df: hdb.labels_.astype(str))
    .query('cluster != "-1"')
    .sort_values(by='cluster')
)

fig = px.scatter(df_umap, x='x', y='y', color='cluster')
fig.show()

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
